# マッチングアルゴリズム

[Top Trading Cycle (TTC)](https://en.wikipedia.org/wiki/Top_trading_cycle)を用いた、物件交換アルゴリズムの実装です。

TTCでは、すべての参加者が、希望順が現状のもの以上の物件に、マッチするような組み合わせが実現できます。アルゴリズムの詳細はリンクをご確認ください。

実際のサービスでは一定期間ごとにマッチングアルゴリズムをバッチ的に実行します。

### 参考文献
https://github.com/MohammadYasinKarbasian/Top-Trading-Cycle/blob/main/top%20trading%20cycle.py

In [ ]:
from copy import deepcopy

In [ ]:
class TopTradingCycle:
    """TTCの実装"""

    def __init__(self):
        self.participants = []  # to store participants' preferences
        self.original_preferences = [] #to store original preferences
        self.is_initialised = False  # to see if the class is initialised
        self.is_matched = False  # to see if the matching is done
        self.final_pairs = []  # to store the results
        self.unmatched_goods = []  # store the goods that haven't been matched yet
        self.unmatched_participants = []  # for unmatched users
        self.edges = []  # to store each participant's current number one preference (preference edge)
        self.next_request = None  # to update the preference edges

    def __str__(self):
        if not self.is_initialised:
            return "\nInitialise participants and goods first.\n"
        elif not self.is_matched:
            return "\nRun the matching algorithm.\n"
        else:
            output = "\nMatched pairs:\n"
            for pair in self.final_pairs:
                output += f"Participant {pair[0]} with Good {pair[1]}.\n"
            return output

    def initialise(self, user_prefs):
        for index, prefs in enumerate(user_prefs):
          self.participants = user_prefs
          self.unmatched_goods = list(range(len(user_prefs)))  # goods are numbered in a 0-based way
          self.unmatched_participants = list(range(len(user_prefs)))
          self.edges = []
          self.is_initialised = True
          self.is_matched = False
          self.next_request = [0 for _ in range(len(self.participants))]

          # initialise edges with each participant's most preferred item, if there's any
          for index in range(len(user_prefs)):
              if user_prefs[index]:  # check if the participant has any preferences left
                  self.edges.append([index, user_prefs[index][0]])
              else:
                  self.edges.append([index, None])  # to deal with cases with no preferences

    def find_cycle(self):
        # find a cycle in the current network of edges using Breadth-First Search (BFS)
        def bfs(node, path):
            def neighbours(n):
                return [e[1] for e in self.edges if e[0] == n]

            nonlocal cycle
            this_path = deepcopy(path)
            for neighbour in neighbours(node):
                if cycle:
                    return
                if neighbour not in visited:
                    visited.append(neighbour)
                    bfs(neighbour, this_path + [neighbour])
                elif neighbour in path:
                    cycle = this_path[this_path.index(neighbour):]

        cycle = []
        visited = []
        for node in self.unmatched_participants:
            if node not in visited:
                visited.append(node)
                bfs(node, [node])
                if cycle:
                    return cycle
        return None

    def match(self):
        """マッチング処理の実行"""
        def tail(node):
            return next((e[1] for e in self.edges if e[0] == node), None)

        def update_edges():
            self.edges = []
            for node in self.unmatched_participants:
                for good in self.participants[node]:
                    if good in self.unmatched_goods:
                        self.edges.append([node, good])
                        break


        while self.unmatched_participants:
            update_edges()
            cycle = self.find_cycle()
            if not cycle:
                break
            for node in cycle:
                matched_good = tail(node)
                if matched_good is not None:
                    self.unmatched_participants.remove(node)
                    self.unmatched_goods.remove(matched_good)
                    self.final_pairs.append((node, matched_good))

        self.is_matched = True

    def add_original_matches(self):
        # add tuples for participants matched with their original items (because they aren't included in the results above)
        all_participants = set(range(0, len(self.participants)))
        matched_participants = set(pair[0] for pair in self.final_pairs)
        missing_participants = all_participants - matched_participants

        # add missing participants with their original items
        for participant in missing_participants:
            self.final_pairs.append((participant, participant))

        # sort the final list
        self.final_pairs.sort(key=lambda pair: pair[0])


In [ ]:
# テストケース1

user_prefs = [
 [5, 0, 2, 1, 3, 9, 7, 6, 4, 8],
 [5, 9, 1, 8, 2, 3, 6, 0, 7, 4],
 [5, 9, 6, 1, 8, 2, 4, 7, 0, 3],
 [5, 9, 8, 2, 4, 6, 7, 3, 0, 1],
 [0, 8, 9, 2, 4, 7, 5, 3, 1, 6],
 [5, 9, 2, 4, 7, 3, 0, 6, 8, 1],
 [5, 6, 0, 7, 2, 3, 8, 1, 4, 9],
 [9, 8, 2, 1, 7, 0, 4, 3, 6, 5],
 [3, 1, 0, 7, 9, 6, 5, 8, 2, 4],
 [5, 9, 0, 6, 3, 8, 2, 1, 4, 7]
 ]


ttc = TopTradingCycle()
ttc.initialise(user_prefs)
ttc.match()
ttc.add_original_matches()
print(ttc.final_pairs)

[(0, 0), (1, 1), (2, 2), (3, 8), (4, 4), (5, 5), (6, 6), (7, 7), (8, 3), (9, 9)]


In [ ]:
# テストケース2

user_prefs = [
    [9, 6, 4, 2, 0],
    [4, 7, 1],
    [8, 5, 0, 4, 7, 1, 9, 6, 2],
    [1, 8, 7, 5, 3],
    [1, 3, 5, 0, 9, 6, 8, 7, 2, 4],
    [6, 2, 7, 4, 0, 9, 3, 5],
    [7, 0, 2, 6],
    [2, 5, 6, 1, 9, 0, 3, 8, 4, 7],
    [2, 7, 1, 9, 3, 5, 0, 8],
    [0, 4, 8, 9]
]

ttc = TopTradingCycle()
ttc.initialise(user_prefs)
ttc.match()
ttc.add_original_matches()
print(ttc.final_pairs)

[(0, 9), (1, 4), (2, 8), (3, 3), (4, 1), (5, 6), (6, 7), (7, 5), (8, 2), (9, 0)]


In [ ]:
# テストケース 3

import random

user_prefs = [random.sample(range(50), 50) for _ in range(50)]
print(user_prefs)

ttc = TopTradingCycle()
ttc.initialise(user_prefs)
ttc.match()
ttc.add_original_matches()
print(ttc.final_pairs)


[[36, 20, 45, 8, 7, 19, 21, 44, 9, 17, 15, 46, 12, 38, 34, 0, 22, 11, 48, 24, 25, 5, 47, 2, 10, 41, 49, 6, 30, 37, 28, 16, 40, 35, 1, 3, 23, 43, 42, 31, 26, 4, 27, 32, 13, 18, 29, 33, 14, 39], [29, 10, 4, 11, 16, 14, 19, 44, 33, 40, 47, 26, 32, 39, 6, 27, 0, 21, 35, 48, 24, 12, 1, 28, 36, 41, 5, 42, 25, 15, 30, 7, 43, 13, 31, 20, 49, 46, 37, 22, 9, 2, 34, 38, 17, 3, 45, 23, 18, 8], [12, 29, 41, 3, 43, 47, 23, 20, 21, 38, 9, 48, 25, 32, 22, 30, 24, 2, 10, 35, 5, 11, 28, 45, 1, 0, 17, 46, 13, 18, 27, 4, 15, 36, 19, 7, 31, 26, 14, 40, 33, 39, 16, 44, 49, 37, 8, 42, 34, 6], [18, 15, 14, 40, 11, 12, 8, 27, 10, 41, 37, 39, 22, 23, 45, 38, 33, 2, 26, 17, 34, 28, 43, 9, 30, 47, 21, 20, 19, 48, 46, 6, 25, 24, 49, 32, 0, 36, 31, 4, 29, 42, 1, 16, 5, 44, 3, 35, 13, 7], [6, 36, 21, 15, 7, 35, 47, 28, 40, 27, 20, 0, 12, 2, 45, 39, 49, 3, 10, 1, 13, 29, 4, 19, 22, 31, 17, 33, 26, 9, 23, 5, 46, 8, 18, 38, 34, 41, 30, 14, 24, 11, 25, 32, 37, 43, 42, 16, 44, 48], [39, 40, 23, 7, 0, 8, 42, 4, 37, 20, 12